In [54]:
# libs
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from scipy import misc
import glob
import random
import cv2
from sklearn.model_selection import train_test_split

# link for KAIST dataset
# https://sites.google.com/site/pedestrianbenchmark/

## Dataset description 

**The KAIST Multispectral Pedestrian Dataset** consists of 95k color-thermal pairs (640x480, 20Hz) taken from a vehicle. All the pairs are manually annotated (person, people, cyclist) for the total of 103,128 dense annotations and 1,182 unique pedestrians.

![title](img/train.PNG)
![title](img/test.PNG)

## Annotations examples

In [73]:
# read annotations from separate txt files to DataFrame
path = '../datasets/KAIST/test_all/annotations/'
true_bboxs = []
for vid_set in sorted(os.listdir(path)):
    for vid in sorted(os.listdir(path+vid_set)):
        for i, k in enumerate(sorted(os.listdir(path+vid_set+'/' + vid))):
            f = open(path+vid_set+'/' +vid + '/' +k,"r")
            bboxs = f.readlines()
            if len(bboxs)>1:
                for bbox in bboxs[1:]:
                    splitted = bbox.split(' ')
                    im_id = k[:-4]
                    cls = splitted[0]
                    x = splitted[1]
                    y = splitted[2]
                    width = splitted[3]
                    height = splitted[4]
                    true_bboxs.append([vid_set+'-'+vid+'-'+im_id, cls, x, y, width, height])

data = pd.DataFrame(true_bboxs, columns=['INDEX', 'CLS', 'X', 'Y', 'W', 'H'])           

In [74]:
data.head(4)

,INDEX,CLS,X,Y,W,H
0,set06-V000-I00029,person,520,224,20,50
1,set06-V000-I00059,person,565,234,25,60
2,set06-V000-I00239,person,433,217,20,35
3,set06-V000-I00269,person,472,215,20,43


In [76]:
converted_annotation_folder = '../mxnet-ssd/data/kaist/annotations/'
images_folder = '../mxnet-ssd/data/kaist/images/'
train_image_set_path = '../mxnet-ssd/data/kaist/image_sets/train.txt'
val_image_set_path = '../mxnet-ssd/data/kaist/image_sets/val.txt'
test_image_set_path = '../mxnet-ssd/data/kaist/image_sets/test.txt'

In [81]:
for INDEX in data.INDEX.unique():
    # find all notes with this index
    index_data = data[data.INDEX == INDEX]
    
    # re-save image
    vid_set, vid, IM_ID = INDEX.split('-')        
    image = misc.imread(('../datasets/KAIST/test_all/images/' + vid_set + '/' + vid +'/' + IM_ID + '.png'))
    
    # save image in dataset images_folder
    np.save(images_folder+INDEX+'.npy', image)
    
    #split data to RGB and TIR for visualization 
    rgb = image[:,:,:3].copy()
    tir = image[:,:,3].copy()

    # add basic annotations info     
    f = open(converted_annotation_folder + INDEX + '.xml','w')
    line = "<annotation>" + '\n'
    f.write(line)
    line = '\t\t<folder>' + "folder" + '</folder>' + '\n'
    f.write(line)
    line = '\t\t<filename>' + INDEX + '</filename>' + '\n'
    f.write(line)
    line = '\t\t<source>\n\t\t<database>Source</database>\n\t</source>\n'
    f.write(line)
    (width, height) = image.shape[0], image.shape[1]
    line = '\t<size>\n\t\t<width>'+ str(width) + '</width>\n\t\t<height>' + str(height) + '</height>\n\t'
    line += '\t<depth>4</depth>\n\t</size>'
    f.write(line)
    line = '\n\t<segmented>Unspecified</segmented>'
    f.write(line)
    
    # get annotations only for this picture
    for idx, row in index_data.iterrows():    
        x=int(row[2])
        y=int(row[3])
        w=int(row[4])
        h=int(row[5])
        line = '\n\t<object>'
        line += '\n\t\t<name>'+row[1]+'</name>\n\t\t<pose>Unspecified</pose>'
        line += '\n\t\t<truncated>Unspecified</truncated>\n\t\t<difficult>0</difficult>'
        line += '\n\t\t<bndbox>\n\t\t\t<xmin>' + str(x) + '</xmin>'            
        line += '\n\t\t\t<ymin>' + str(y) + '</ymin>'
        line += '\n\t\t\t<xmax>' + str(x+w) + '</xmax>'
        line += '\n\t\t\t<ymax>' + str(y+h) + '</ymax>'
        line += '\n\t\t</bndbox>'
        line += '\n\t</object>\n' 
        f.write(line) 
        
        cv2.rectangle(rgb,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.rectangle(tir,(x,y),(x+w,y+h),(0,255,0),2) 
    line = "</annotation>" + '\n'
    f.write(line)
    f.close()
    
    # create/save figures with thermal-rgb pairs
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(20, 20))
    ax = axes.ravel()
    ax[0].imshow(rgb)
    ax[0].set_title('RGB')
    ax[0].axis('off')
    ax[1].imshow(tir, cmap=plt.get_cmap('Spectral_r'))
    ax[1].set_title('TIR')
    ax[1].axis('off')
    fig.tight_layout()
    fig.savefig('../figures/'+INDEX+'.png')
    plt.close(fig)

In [61]:
# create train and val image sets
train_index, val_index = train_test_split(data.INDEX.unique(), test_size = 0.08)

In [64]:
save_image_set_txt(train_image_set_path, index)
save_image_set_txt(val_image_set_path, val_index)
save_image_set_txt(test_image_set_path, data.INDEX.unique())

In [72]:
# useful functions 
def save_image_set_txt(path, index):
    f = open(path, 'w')
    for ind in index:
        f.write(ind+'\n')
    f.close()